In [12]:
# import sys
# sys.executable # this should be same in jupyter and command line

import argparse
import torch
from pathlib import Path
import numpy as np
from pcdet.utils import common_utils
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets.kitti.meshed_kitti_dataset import MeshedKittiDataset
from pcdet.datasets.baraja.meshed_baraja_dataset import MeshedBarajaDataset
from pcdet.datasets.waymo.meshed_waymo_dataset import MeshedWaymoDataset
from pcdet.datasets.nuscenes.meshed_nuscenes_dataset import MeshedNuScenesDataset
from pcdet.datasets.nuscenes.nuscenes_dataset import NuScenesDataset

# cfg_file = '/SEE-MTDA/detector/output/source-waymo/secondiou/see/secondiou_ros_custom1000_GM-ORH005/default/secondiou_ros_custom1000_GM-ORH005_eval-nusc4025.yaml'
cfg_file = '/SEE-MTDA/detector/output/source-waymo/secondiou/see-v2/secondiou_ros_custom1000_PCN-norm-coarse/default/secondiou_ros_custom1000_PCN-norm-coarse.yaml'
# cfg_file = '/SEE-MTDA/detector/output/source-waymo/secondiou/baseline/secondiou_custom1000/default/secondiou_custom1000_eval-nusc4025.yaml'
cfg_from_yaml_file(cfg_file, cfg)

__all__ = {
    'NuScenesDataset': NuScenesDataset,
    'MeshedNuScenesDataset': MeshedNuScenesDataset,
    'MeshedKittiDataset': MeshedKittiDataset,
    'MeshedWaymoDataset': MeshedWaymoDataset,
    'MeshedBarajaDataset': MeshedBarajaDataset
}

cfg.DATA_CONFIG_TAR.FILTER_MIN_POINTS_IN_GT = 5
print(f'Reading from {cfg.DATA_CONFIG_TAR._BASE_CONFIG_}')
print(f'Using pkl {cfg.DATA_CONFIG_TAR.INFO_PATH["test"]}')
logger = common_utils.create_logger(Path('jupy_log.txt'), rank=cfg.LOCAL_RANK)
dataset = __all__[cfg.DATA_CONFIG_TAR.DATASET](
        dataset_cfg=cfg.DATA_CONFIG_TAR,
        class_names=cfg.DATA_CONFIG_TAR.CLASS_NAMES,
        root_path=None,
        training=False,
        logger=logger,
    )

# cfg.DATA_CONFIG.FILTER_MIN_POINTS_IN_GT = 5
# print(f'Using pkl {cfg.DATA_CONFIG.INFO_PATH["test"]}')
# logger = common_utils.create_logger(Path('jupy_log.txt'), rank=cfg.LOCAL_RANK)
# dataset = __all__[cfg.DATA_CONFIG.DATASET](
#         dataset_cfg=cfg.DATA_CONFIG,
#         class_names=cfg.CLASS_NAMES,
#         root_path=None,
#         training=False,
#         logger=logger,
#     )

[2022-05-10 21:46:16,091  meshed_kitti_dataset.py 37  INFO]  Loading MeshedKittiDataset dataset
[2022-05-10 21:46:16,091  meshed_kitti_dataset.py 37  INFO]  Loading MeshedKittiDataset dataset
[2022-05-10 21:46:16,091  meshed_kitti_dataset.py 37  INFO]  Loading MeshedKittiDataset dataset
[2022-05-10 21:46:16,091  meshed_kitti_dataset.py 37  INFO]  Loading MeshedKittiDataset dataset
INFO - 2022-05-10 21:46:16,091 - meshed_kitti_dataset - Loading MeshedKittiDataset dataset


Reading from cfgs/dataset_configs/da_kitti_dataset_meshed.yaml
Using pkl ['infos_meshed_KIT-GM-EXT-PCN-NORMSCALE-COARSE/kitti_infos_val.pkl']


[2022-05-10 21:46:16,271  meshed_kitti_dataset.py 51  INFO]  Total samples for MeshedKittiDataset dataset: 3769
[2022-05-10 21:46:16,271  meshed_kitti_dataset.py 51  INFO]  Total samples for MeshedKittiDataset dataset: 3769
[2022-05-10 21:46:16,271  meshed_kitti_dataset.py 51  INFO]  Total samples for MeshedKittiDataset dataset: 3769
[2022-05-10 21:46:16,271  meshed_kitti_dataset.py 51  INFO]  Total samples for MeshedKittiDataset dataset: 3769
INFO - 2022-05-10 21:46:16,271 - meshed_kitti_dataset - Total samples for MeshedKittiDataset dataset: 3769


In [13]:
import open3d as o3d

def convert_to_o3dpcd(points, color=None):
    if type(points) == list:
        pcds = []
        for pointcloud in points:
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(pointcloud[:,:3])
            pcds.append(pcd)
        return pcds
    else:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points[:,:3])
        if color:
            pcd.paint_uniform_color(color)
        return pcd

def opd_to_boxpts(box):
    """
    Takes an array containing [x,y,z,l,w,h,r], and returns an [8, 3] matrix that 
    represents the [x, y, z] for each 8 corners of the box.
    
    Note: Openpcdet __getitem__ gt_boxes are in the format [x,y,z,l,w,h,r,alpha]
    where alpha is "observation angle of object, ranging [-pi..pi]"
    """
    # To return
    corner_boxes = np.zeros((8, 3))

    translation = box[0:3]
    l, w, h = box[3], box[4], box[5] # waymo, nusc, kitti is all l,w,h after OpenPCDet processing
    rotation = box[6]

    # Create a bounding box outline
    bounding_box = np.array([[l/2, w/2, h/2],
                             [l/2, -w/2, h/2],
                             [-l/2, w/2, h/2],
                             [-l/2, -w/2, h/2],
                             [l/2, w/2, -h/2],
                             [l/2, -w/2, -h/2],
                             [-l/2, w/2, -h/2],
                             [-l/2, -w/2, -h/2]])

    # Standard 3x3 rotation matrix around the Z axis
    rotation_matrix = np.array([
        [np.cos(rotation), np.sin(rotation), 0.0],
        [-np.sin(rotation), np.cos(rotation), 0.0],
        [0.0, 0.0, 1.0]])
    vcbox = bounding_box @ rotation_matrix
    vcbox += box[:3]
    
    return vcbox

def boxpts_to_o3dbox(box_pts, colour=None):
    boxpts = o3d.utility.Vector3dVector(box_pts)
    o3dbox = o3d.geometry.OrientedBoundingBox().create_from_points(boxpts)
    if colour is None:
        colour = [1,0,0]
        
    o3dbox.color = np.array(colour)
    return o3dbox


In [14]:
def text_3d(text, pos, direction=None, degree=90.0, density=10, font='/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', font_size=16):
    """
    Code from https://github.com/isl-org/Open3D/issues/2
    
    Generate a 3D text point cloud used for visualization.
    :param text: content of the text
    :param pos: 3D xyz position of the text upper left corner
    :param direction: 3D normalized direction of where the text faces
    :param degree: in plane rotation of text
    :param font: Name of the font - change it according to your system
    :param font_size: size of the font
    :return: o3d.geoemtry.PointCloud object
    """
    if direction is None:
        direction = (1., 0., 0.)

    from PIL import Image, ImageFont, ImageDraw
    from pyquaternion import Quaternion

    font_obj = ImageFont.truetype(font, font_size * density)
    font_dim = font_obj.getsize(text)

    img = Image.new('RGB', font_dim, color=(255, 255, 255))
    draw = ImageDraw.Draw(img)
    draw.text((0, 0), text, font=font_obj, fill=(0, 0, 0))
    img = np.asarray(img)
    img_mask = img[:, :, 0] < 128
    indices = np.indices([*img.shape[0:2], 1])[:, img_mask, 0].reshape(3, -1).T

    pcd = o3d.geometry.PointCloud()
    pcd.colors = o3d.utility.Vector3dVector(img[img_mask, :].astype(float) / 255.0)
    pcd.points = o3d.utility.Vector3dVector(indices / 1000 / density)

    raxis = np.cross([0.0, 0.0, 1.0], direction)
    if np.linalg.norm(raxis) < 1e-6:
        raxis = (0.0, 0.0, 1.0)
    trans = (Quaternion(axis=raxis, radians=np.arccos(direction[2])) *
             Quaternion(axis=direction, degrees=degree)).transformation_matrix
    trans[0:3, 3] = np.asarray(pos)
    pcd.transform(trans)
    return pcd

In [15]:
in_dict = dataset.__getitem__(47) # 1637, 167

opcd = convert_to_o3dpcd(in_dict['points'])
o3dboxes = [boxpts_to_o3dbox(opd_to_boxpts(box)) for box in in_dict['gt_boxes']]

objs = [opcd.crop(o3dbox) for o3dbox in o3dboxes]
text_loc = [o3dbox.center + np.array([0,0,o3dbox.extent[2]]) for o3dbox in o3dboxes]
num_pts = [len(obj.points) for obj in objs]
labels = [text_3d(str(npt), loc, font_size=500, density=1) for npt, loc in zip(num_pts, text_loc)]

o3d.visualization.draw_geometries([opcd] + o3dboxes + labels)

In [5]:
def mask_points_by_range(points, limit_range):
    # Input: [xmin, ymin, zmin, xmax, ymax, zmax] 
    # Though this only limits x and y
    mask = (points[:, 0] >= limit_range[0]) & (points[:, 0] <= limit_range[3]) \
           & (points[:, 1] >= limit_range[1]) & (points[:, 1] <= limit_range[4])
    return mask

LIMITS = [-24, -24, -2, 24, 24, 4]
pts = in_dict['points'][mask_points_by_range(in_dict['points'], LIMITS)]
limpcd = convert_to_o3dpcd(pts, [0.9,0.9,0])
o3d.visualization.draw_geometries([opcd, limpcd] + o3dboxes + labels)

In [38]:
import copy 

index = 213
info = copy.deepcopy(dataset.infos[index])
points = dataset.get_meshed_lidar(index)

# Shift points 
if dataset.dataset_cfg.get('SHIFT_COOR', None):
    points[:, 0:3] += np.array(dataset.dataset_cfg.SHIFT_COOR, dtype=np.float32)

input_dict = {
    'points': points,
    'frame_id': Path(info['meshed_lidar_path']).stem,
    'metadata': {'token': info['token']},
    'num_lidar_pts': info['num_lidar_pts'],
    'num_meshed_lidar_pts': info['num_meshed_lidar_pts'],
    'gt_names': info['gt_names']
}

if 'gt_boxes' in info:
    if dataset.dataset_cfg.get('FILTER_MIN_POINTS_IN_GT', False):
        mask = (info['num_lidar_pts'] > dataset.dataset_cfg.FILTER_MIN_POINTS_IN_GT)
    else:
        mask = None

    input_dict.update({
        'gt_names': info['gt_names'] if mask is None else info['gt_names'][mask],
        'gt_boxes': info['gt_boxes'] if mask is None else info['gt_boxes'][mask],
        'num_lidar_pts': info['num_lidar_pts'] if mask is None else info['num_lidar_pts'][mask],
        'num_meshed_lidar_pts': info['num_meshed_lidar_pts'] if mask is None else info['num_meshed_lidar_pts'][mask]
    })
    
print(input_dict)

{'points': array([[ -3.4035199 , -12.885465  ,   0.0545764 ],
       [ -3.665458  , -12.484063  ,   0.10108173],
       [ -3.8360603 , -12.696234  ,   0.6848511 ],
       ...,
       [-19.87915   ,  -0.20320109,   3.9118257 ],
       [-19.831064  ,  -0.19266169,   4.844988  ],
       [-19.822323  ,  -0.18565044,   5.321441  ]], dtype=float32), 'frame_id': 'f552ec59c267469899b3472d6bd9bf1d#000213', 'metadata': {'token': 'f552ec59c267469899b3472d6bd9bf1d'}, 'num_lidar_pts': array([1665,   11,    4,   10,    5,    4,   18,    5,    3,    3,    7,
          4,  116,    3,   11,    6,    4,   15,    3,  143,   40,   10,
          9,   37,    3,    8]), 'num_meshed_lidar_pts': array([4420,   22,   13,   20,    9,   10,   37,    8,    5,    7,   12,
          4, 1020,    5,   21,   11,    8,   30,   10, 1018,  441,   21,
         17,  499,    6,   18]), 'gt_names': array(['car', 'motorcycle', 'bicycle', 'car', 'car', 'car', 'car', 'car',
       'car', 'pedestrian', 'motorcycle', 'car', 'car',

# Inference

In [18]:
from pcdet.datasets import build_dataloader
from pcdet.models import build_network

ckpt_path = '/SEE-MTDA/detector/output/source-waymo/secondiou/see-v2/secondiou_ros_custom1000_PCN-norm-coarse/default/ckpt/checkpoint_epoch_82.pth'
# ckpt_path = '/SEE-MTDA/model_zoo/waymo_secondiou_baseline_1192.pth'

test_set, test_loader, sampler = build_dataloader(
            dataset_cfg=cfg.DATA_CONFIG_TAR,
            class_names=cfg.DATA_CONFIG_TAR.CLASS_NAMES,
            batch_size=1,
            dist=False, workers=2, logger=logger, training=False
        )

[2022-05-10 21:47:39,524  meshed_kitti_dataset.py 37  INFO]  Loading MeshedKittiDataset dataset
[2022-05-10 21:47:39,524  meshed_kitti_dataset.py 37  INFO]  Loading MeshedKittiDataset dataset
[2022-05-10 21:47:39,524  meshed_kitti_dataset.py 37  INFO]  Loading MeshedKittiDataset dataset
[2022-05-10 21:47:39,524  meshed_kitti_dataset.py 37  INFO]  Loading MeshedKittiDataset dataset
INFO - 2022-05-10 21:47:39,524 - meshed_kitti_dataset - Loading MeshedKittiDataset dataset
[2022-05-10 21:47:39,677  meshed_kitti_dataset.py 51  INFO]  Total samples for MeshedKittiDataset dataset: 3769
[2022-05-10 21:47:39,677  meshed_kitti_dataset.py 51  INFO]  Total samples for MeshedKittiDataset dataset: 3769
[2022-05-10 21:47:39,677  meshed_kitti_dataset.py 51  INFO]  Total samples for MeshedKittiDataset dataset: 3769
[2022-05-10 21:47:39,677  meshed_kitti_dataset.py 51  INFO]  Total samples for MeshedKittiDataset dataset: 3769
INFO - 2022-05-10 21:47:39,677 - meshed_kitti_dataset - Total samples for Mes

In [59]:
mypath = '/SEE-MTDA/detector/output/source-waymo/secondiou/see-v2/secondiou_ros_custom1000_PCN-norm-coarse/default/ckpt/ckpt-best.pth'
'ckpt-best' in mypath

True

In [56]:
Path(ckpt_path).exists()

True

In [48]:
state_dict = torch.load(ckpt_path)

In [55]:
state_dict.keys()

dict_keys(['epoch', 'it', 'model_state', 'optimizer_state', 'version'])

In [19]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
model.load_params_from_file(filename=ckpt_path, logger=logger, to_cpu=False)
model.cuda()
model.eval()

dataset = test_loader.dataset
class_names = dataset.class_names

[2022-05-10 21:47:39,828  detector3d_template.py 325  INFO]  ==> Loading parameters from checkpoint /SEE-MTDA/detector/output/source-waymo/secondiou/see-v2/secondiou_ros_custom1000_PCN-norm-coarse/default/ckpt/checkpoint_epoch_82.pth to GPU
[2022-05-10 21:47:39,828  detector3d_template.py 325  INFO]  ==> Loading parameters from checkpoint /SEE-MTDA/detector/output/source-waymo/secondiou/see-v2/secondiou_ros_custom1000_PCN-norm-coarse/default/ckpt/checkpoint_epoch_82.pth to GPU
[2022-05-10 21:47:39,828  detector3d_template.py 325  INFO]  ==> Loading parameters from checkpoint /SEE-MTDA/detector/output/source-waymo/secondiou/see-v2/secondiou_ros_custom1000_PCN-norm-coarse/default/ckpt/checkpoint_epoch_82.pth to GPU
[2022-05-10 21:47:39,828  detector3d_template.py 325  INFO]  ==> Loading parameters from checkpoint /SEE-MTDA/detector/output/source-waymo/secondiou/see-v2/secondiou_ros_custom1000_PCN-norm-coarse/default/ckpt/checkpoint_epoch_82.pth to GPU
INFO - 2022-05-10 21:47:39,828 - det

In [31]:
from pcdet.models import load_data_to_gpu

def statistics_info(cfg, ret_dict, metric, disp_dict):
    for cur_thresh in cfg.MODEL.POST_PROCESSING.RECALL_THRESH_LIST:
        metric['recall_roi_%s' % str(cur_thresh)] += ret_dict.get('roi_%s' % str(cur_thresh), 0)
        metric['recall_rcnn_%s' % str(cur_thresh)] += ret_dict.get('rcnn_%s' % str(cur_thresh), 0)
    metric['gt_num'] += ret_dict.get('gt', 0)
    min_thresh = cfg.MODEL.POST_PROCESSING.RECALL_THRESH_LIST[0]
    disp_dict['recall_%s' % str(min_thresh)] = \
        '(%d, %d) / %d' % (metric['recall_roi_%s' % str(min_thresh)], metric['recall_rcnn_%s' % str(min_thresh)], metric['gt_num'])

metric = {
    'gt_num': 0,
}
det_annos = []
for cur_thresh in cfg.MODEL.POST_PROCESSING.RECALL_THRESH_LIST:
    metric['recall_roi_%s' % str(cur_thresh)] = 0
    metric['recall_rcnn_%s' % str(cur_thresh)] = 0


for i, batch_dict in enumerate(test_loader):
    if i < 20:
        load_data_to_gpu(batch_dict)
        with torch.no_grad():
            pred_dicts, ret_dict = model(batch_dict)
        disp_dict = {}

        statistics_info(cfg, ret_dict, metric, disp_dict)
        annos = dataset.generate_prediction_dicts(
            batch_dict, pred_dicts, class_names,
            output_path=None
        )
        det_annos += annos

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:3448: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


In [44]:
import pickle

with open('/SEE-MTDA/detector/output/source-nuscenes/secondiou/see-v2/secondiou_ros_PCN-norm-coarse/eval/eval_all_default/default/epoch_134/val/result_dict.pkl', 'rb') as f:
    res = pickle.load(f)
# 'Car_3d/moderate_R40'

In [45]:
res

{'Car_aos/easy_R40': 96.96647654132234,
 'Car_aos/moderate_R40': 94.52864579283711,
 'Car_aos/hard_R40': 97.02194298208322,
 'Car_3d/easy_R40': 67.38243672543798,
 'Car_3d/moderate_R40': 61.75487685752435,
 'Car_3d/hard_R40': 62.53479497279033,
 'Car_bev/easy_R40': 89.30661533108474,
 'Car_bev/moderate_R40': 85.65395898111842,
 'Car_bev/hard_R40': 86.53882710036704,
 'Car_image/easy_R40': 96.97348182138954,
 'Car_image/moderate_R40': 94.54188147375116,
 'Car_image/hard_R40': 97.0590166158118}

In [43]:
ret

{'recall/roi_0.3': 0.9610598434004475,
 'recall/rcnn_0.3': 0.9610598434004475,
 'recall/roi_0.5': 0.9525307606263982,
 'recall/rcnn_0.5': 0.9525307606263982,
 'recall/roi_0.7': 0.7220357941834452,
 'recall/rcnn_0.7': 0.7220357941834452,
 'Car_aos/easy_R40': 96.96647654132234,
 'Car_aos/moderate_R40': 94.52864579283711,
 'Car_aos/hard_R40': 97.02194298208322,
 'Car_3d/easy_R40': 67.38243672543798,
 'Car_3d/moderate_R40': 61.75487685752435,
 'Car_3d/hard_R40': 62.53479497279033,
 'Car_bev/easy_R40': 89.30661533108474,
 'Car_bev/moderate_R40': 85.65395898111842,
 'Car_bev/hard_R40': 86.53882710036704,
 'Car_image/easy_R40': 96.97348182138954,
 'Car_image/moderate_R40': 94.54188147375116,
 'Car_image/hard_R40': 97.0590166158118}

In [28]:
in_dict = dataset.__getitem__(i) # 1637, 167

gt_boxes = batch_dict['gt_boxes'].squeeze(0).cpu().numpy()
pred_boxes = pred_dicts[0]['pred_boxes'].cpu().numpy()

o3d_gtb = [boxpts_to_o3dbox(opd_to_boxpts(box),[0,0,1]) for box in gt_boxes]
o3d_pb = [boxpts_to_o3dbox(opd_to_boxpts(box),[0,0.7,0]) for box in pred_boxes]
opcd = convert_to_o3dpcd(in_dict['points'])

o3d.visualization.draw_geometries(o3d_gtb + o3d_pb + [opcd])

In [22]:
in_dict = dataset.__getitem__(i) # 1637, 167

opcd = convert_to_o3dpcd(in_dict['points'])
o3dboxes = [boxpts_to_o3dbox(opd_to_boxpts(box)) for box in in_dict['gt_boxes']]

objs = [opcd.crop(o3dbox) for o3dbox in o3dboxes]
text_loc = [o3dbox.center + np.array([0,0,o3dbox.extent[2]]) for o3dbox in o3dboxes]
num_pts = [len(obj.points) for obj in objs]
labels = [text_3d(str(npt), loc, font_size=500, density=1) for npt, loc in zip(num_pts, text_loc)]

o3d.visualization.draw_geometries([opcd] + o3dboxes + labels)

In [5]:
import pickle

for info_path in dataset.dataset_cfg.INFO_PATH['test']:
    info_path = dataset.root_path / info_path
    print(f'Reading from {info_path}')
    if not info_path.exists():
        continue
    with open(info_path, 'rb') as f:
        infos = pickle.load(f)

Reading from ../../data/nuscenes/custom_t4025-v3980/infos_meshed_NUS-DM-ORH005/nuscenes_infos_2sweeps_val.pkl


In [10]:
from tqdm import tqdm 
import open3d as o3d

updated_infos = []

for i in tqdm(range(len(infos)), total=len(infos)):
    info = infos[i]
    meshed_lidar_file = dataset.root_path / info['meshed_lidar_path']
    opcd = o3d.io.read_point_cloud(str(meshed_lidar_file))
    gt_boxes = info['gt_boxes']
    o3dboxes = [boxpts_to_o3dbox(opd_to_boxpts(box)) for box in gt_boxes]
    objs = [opcd.crop(o3dbox) for o3dbox in o3dboxes]    
    num_pts = [len(obj.points) for obj in objs]
#     text_loc = [o3dbox.center + np.array([0,0,o3dbox.extent[2]]) for o3dbox in o3dboxes]
#     labels = [text_3d(str(npt), loc, font_size=500, density=1) for npt, loc in zip(num_pts, text_loc)]
    info['num_meshed_lidar_pts'] = np.array(num_pts)
    updated_infos.append(info)
    
    

100%|███████████████████████████████████████████| 3980/3980 [01:02<00:00, 63.33it/s]


In [11]:
toutput = open(str(Path(dataset.dataset_cfg.DATA_PATH).absolute() / 'infos_meshed_NUS-DM-ORH005' / f'nuscenes_infos_2sweeps_val_meshednum.pkl'), 'wb')
pickle.dump(updated_infos, toutput)

In [17]:
o3d.visualization.draw_geometries([opcd] + o3dboxes )

In [20]:
for d in zip(info['gt_names'], info['num_lidar_pts'], info['num_meshed_lidar_pts']):
    print(d)

('car', 3, 6)
('traffic_cone', 19, 38)
('car', 63, 461)
('car', 18, 32)
('barrier', 2, 4)
('pedestrian', 5, 12)
('car', 4, 7)
('pedestrian', 19, 35)
('car', 10, 23)
('car', 1, 4)
('car', 5, 9)
('car', 2, 3)
('car', 8, 15)
('traffic_cone', 9, 17)
('pedestrian', 5, 10)
('barrier', 6, 13)
('traffic_cone', 2, 3)
('truck', 8, 15)
('car', 7, 17)
('car', 19, 49)
('car', 16, 32)
('car', 1, 2)
('car', 4, 9)
('pedestrian', 1, 8)
('barrier', 3, 6)
('car', 2, 4)
('car', 3, 10)
('car', 8, 20)
('barrier', 6, 14)
('pedestrian', 5, 10)
('pedestrian', 5, 10)
('ignore', 8, 16)
('car', 6, 10)
('car', 10, 22)
('car', 2, 4)
('barrier', 25, 50)
('barrier', 27, 50)
('car', 9, 17)
('car', 1, 2)
('car', 1, 4)
('traffic_cone', 4, 6)
('car', 3, 8)
('pedestrian', 12, 23)
('car', 2, 4)
('car', 4, 8)
('car', 30, 322)
('car', 5, 9)
('barrier', 15, 31)
('barrier', 8, 16)
('car', 2, 4)
('barrier', 11, 21)
('car', 1, 2)
('pedestrian', 4, 9)
('car', 2, 5)
('car', 99, 678)
('car', 2, 4)
('car', 2, 4)
('car', 596, 2117)
(